# Furuta Pendulum Simulation Environment

This notebook provides an interactive simulation environment for the Rotary Inverted Pendulum.

**What this does:**
- Implements the exact nonlinear dynamics from your mathematical model
- Uses the same discrete-time controller as your firmware (200 Hz)
- Allows you to tune gains and test stability before hardware

## Quick Start
1. Run all cells in Section 1 (Setup)
2. Run Section 2 to verify model parameters
3. Run Section 3 for basic simulation
4. Use Section 4 for gain tuning

## 1. Setup and Imports

In [ ]:
import sys
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import HTML

# Add simulation directory to path
sys.path.insert(0, '.')

from furuta_model import FurutaParams, FurutaDynamics
from controller import ControllerParams, FurutaController
from simulator import FurutaSimulator, SimulationConfig, SimulationResult
from visualize import plot_simulation_result, quick_plot, animate_pendulum, compare_simulations

%matplotlib inline
plt.rcParams['figure.dpi'] = 100
print("✓ All modules loaded successfully!")

## 2. Model Verification

Let's verify that the simulation model matches your mathematical derivation.

In [ ]:
# Create model with your hardware parameters
params = FurutaParams()
model = FurutaDynamics(params)

print("="*60)
print("PHYSICAL PARAMETERS (from your hardware)")
print("="*60)
print(f"")
print(f"Arm length L_r = {params.L_r*1000:.1f} mm")
print(f"Pendulum length L_p = {params.L_p*1000:.1f} mm")
print(f"Pendulum mass m_p = {params.m_p*1000:.2f} g")
print(f"Pendulum COM l_p = {params.l_p*1000:.1f} mm")
print(f"")
print(f"Ĵ₀ (arm + pend at tip) = {params.J_hat_0:.6e} kg·m²")
print(f"Ĵ₁ (pendulum @ pivot) = {params.J_1:.6e} kg·m²")
print(f"K (coupling) = {params.K:.6e} kg·m²")
print(f"G (gravity) = {params.G:.6e} N·m")
print(f"")
print("="*60)
print("DERIVED QUANTITIES")
print("="*60)
print(f"")
print(f"Natural frequency ω = {model.natural_frequency():.2f} rad/s = {model.natural_frequency()/(2*np.pi):.2f} Hz")
print(f"Fall time constant τ = {model.fall_time_constant()*1000:.1f} ms")
print(f"")
print("Open-loop poles:")
poles = model.get_open_loop_poles()
for p in poles:
    print(f"  λ = {p:.3f}")
print(f"")
print("Note: Positive real pole ≈ +10 confirms unstable equilibrium!")

In [ ]:
# Compare with your mathematical modelling notes
print("\n" + "="*60)
print("COMPARISON WITH YOUR CALCULATIONS")
print("="*60)
print(f"")
print("From your Handwritten_Notes.md:")
print(f"  A = G/J₁ = 100.8 s⁻²")
print(f"  B = K/J₁ = 1.952")
print(f"")
print("Simulation model:")
A_sim = params.G / params.J_1
B_sim = params.K / params.J_1
print(f"  A = G/J₁ = {A_sim:.1f} s⁻²")
print(f"  B = K/J₁ = {B_sim:.3f}")
print(f"")
print("Your controller gains (in step units):")
print(f"  ACC_KP = 742 steps/s²/deg")
print(f"  ACC_KD = 54.6 steps/s²/(deg/s)")
print(f"")
print("These give closed-loop poles:")
print(f"  ω_c = 15 rad/s, ζ = 0.8 (from your pole placement)")

## 3. Basic Simulation Test

Run a simulation with your current controller gains to verify it works.

In [ ]:
# Configure simulation
config = SimulationConfig(
    duration_s=5.0,           # 5 seconds
    alpha_0_deg=5.0,          # Start 5° from upright
    theta_0_deg=0.0,          # Arm centered
    engage_delay_s=0.05,      # Quick engage
)

# Use your current hardware gains
ctrl_params = ControllerParams(
    ACC_KP=742.0,
    ACC_KD=54.6,
    ACC_KI=0.0,
    KTHETA=15.0,
    KTHETADOT=60.0,
    outer_loop_enabled=True,
    VEL_LEAK=2.0,
)

# Run simulation
sim = FurutaSimulator(ctrl_params=ctrl_params, config=config)
result = sim.run(verbose=True)

In [ ]:
# Plot results
if result.success:
    fig = plot_simulation_result(result, "Simulation with Current Hardware Gains")
    plt.show()
else:
    print(f"Simulation failed! Pendulum fell at t={result.fall_time:.3f}s")
    quick_plot(result)
    plt.show()

## 4. Gain Tuning

Test different gain combinations to find the optimal values.

In [ ]:
# Define gain ranges to test
# Based on your mathematical model: KP ≈ 742, KD ≈ 54.6

kp_values = [400, 500, 600, 742, 850, 1000, 1200]
kd_values = [30, 40, 50, 54.6, 60, 70, 80]

print("Testing gain combinations...")
print("(This may take a minute)")
print("")

results_dict = {}
for kp in kp_values:
    for kd in kd_values:
        ctrl = ControllerParams(ACC_KP=kp, ACC_KD=kd, outer_loop_enabled=True)
        cfg = SimulationConfig(duration_s=3.0, alpha_0_deg=5.0, engage_delay_s=0.05)
        
        sim = FurutaSimulator(ctrl_params=ctrl, config=cfg)
        result = sim.run(verbose=False)
        
        results_dict[(kp, kd)] = {
            'success': result.success,
            'fall_time': result.fall_time,
            'max_alpha': result.max_alpha,
            'max_theta': result.max_theta
        }
        
        status = "✓" if result.success else f"✗ (fell at {result.fall_time:.2f}s)"
        print(f"KP={kp:6.0f}, KD={kd:5.1f}: {status}")

In [ ]:
# Create stability heatmap
from visualize import plot_gain_stability_map

fig = plot_gain_stability_map(results_dict, kp_values, kd_values)
plt.show()

## 5. Compare Different Configurations

In [ ]:
# Compare: outer loop ON vs OFF
config = SimulationConfig(duration_s=3.0, alpha_0_deg=3.0, engage_delay_s=0.05)

# With outer loop
ctrl_on = ControllerParams(ACC_KP=742, ACC_KD=54.6, outer_loop_enabled=True)
sim_on = FurutaSimulator(ctrl_params=ctrl_on, config=config)
result_on = sim_on.run(verbose=False)

# Without outer loop  
ctrl_off = ControllerParams(ACC_KP=742, ACC_KD=54.6, outer_loop_enabled=False)
sim_off = FurutaSimulator(ctrl_params=ctrl_off, config=config)
result_off = sim_off.run(verbose=False)

print(f"With outer loop: {'SUCCESS' if result_on.success else 'FAIL'}")
print(f"Without outer loop: {'SUCCESS' if result_off.success else 'FAIL'}")

fig = compare_simulations(
    [result_on, result_off],
    ['Outer Loop ON', 'Outer Loop OFF'],
    'Effect of Outer Loop on Stability'
)
plt.show()

## 6. Disturbance Rejection Test

Test how well the controller recovers from a sudden perturbation.

In [ ]:
# Simulate a disturbance at t=1s
config = SimulationConfig(
    duration_s=4.0,
    alpha_0_deg=1.0,              # Start near upright
    engage_delay_s=0.05,
    disturbance_time_s=1.0,       # Apply disturbance at t=1s
    disturbance_alpha_deg=10.0,   # 10 degree kick
)

ctrl = ControllerParams(ACC_KP=742, ACC_KD=54.6, outer_loop_enabled=True)
sim = FurutaSimulator(ctrl_params=ctrl, config=config)
result = sim.run(verbose=True)

fig = plot_simulation_result(result, "Disturbance Rejection Test (10° kick at t=1s)")
plt.show()

## 7. Sensor Noise Test

Test robustness to sensor noise (simulating AS5600 resolution).

In [ ]:
# AS5600 has 12-bit resolution = 360°/4096 ≈ 0.088°
# Add some noise to test robustness

config_clean = SimulationConfig(
    duration_s=3.0,
    alpha_0_deg=3.0,
    engage_delay_s=0.05,
    sensor_noise_alpha_deg=0.0,
)

config_noisy = SimulationConfig(
    duration_s=3.0,
    alpha_0_deg=3.0,
    engage_delay_s=0.05,
    sensor_noise_alpha_deg=0.1,  # 0.1° std dev
)

ctrl = ControllerParams(ACC_KP=742, ACC_KD=54.6, outer_loop_enabled=True)

sim_clean = FurutaSimulator(ctrl_params=ctrl, config=config_clean)
result_clean = sim_clean.run(verbose=False)

sim_noisy = FurutaSimulator(ctrl_params=ctrl, config=config_noisy)
result_noisy = sim_noisy.run(verbose=False)

print(f"Without noise: {'SUCCESS' if result_clean.success else 'FAIL'}")
print(f"With 0.1° noise: {'SUCCESS' if result_noisy.success else 'FAIL'}")

fig = compare_simulations(
    [result_clean, result_noisy],
    ['No Noise', '0.1° Sensor Noise'],
    'Effect of Sensor Noise'
)
plt.show()

## 8. Animation (Optional)

Create an animation of the pendulum motion.

In [ ]:
# Run a simulation for animation
config = SimulationConfig(
    duration_s=3.0,
    alpha_0_deg=5.0,
    engage_delay_s=0.05,
)

ctrl = ControllerParams(ACC_KP=742, ACC_KD=54.6, outer_loop_enabled=True)
sim = FurutaSimulator(ctrl_params=ctrl, config=config)
result = sim.run(verbose=False)

if result.success:
    print("Creating animation... (this may take a moment)")
    anim, fig = animate_pendulum(result, fps=30, speed=0.5)
    plt.show()
else:
    print("Simulation failed, cannot animate.")

## 9. Export Results

Save simulation data in the same format as hardware logs for comparison.

In [ ]:
# Save to CSV (same format as firmware logs)
if result.success:
    filename = "simulation_result.csv"
    result.save_csv(filename)
    print(f"Saved simulation data to {filename}")
    print("\nFirst 10 lines:")
    print(result.to_csv_string().split('\n')[:11])

## 10. Custom Experiments

Use this section to run your own experiments.

In [ ]:
# Example: Test with different arm lengths
# Uncomment and modify as needed

# results = []
# labels = []
# 
# for L_r in [0.15, 0.19, 0.25]:
#     params = FurutaParams(L_r=L_r)
#     ctrl = ControllerParams(ACC_KP=742, ACC_KD=54.6)
#     config = SimulationConfig(duration_s=3.0, alpha_0_deg=5.0)
#     
#     sim = FurutaSimulator(model_params=params, ctrl_params=ctrl, config=config)
#     result = sim.run(verbose=False)
#     
#     results.append(result)
#     labels.append(f"L_r = {L_r*1000:.0f} mm")
# 
# compare_simulations(results, labels, "Effect of Arm Length")
# plt.show()